In [3]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import math
import pandas as pd

### web-scraping code

In [162]:
# "apartments" or 'houses'

properties_to_look_for = "apartments"

In [163]:
def url_selection(properties_to_look_for):

    if properties_to_look_for == 'apartments':
        url = 'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
#         url_several = f'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    elif properties_to_look_for == 'houses':
        url = 'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
#         url_several = f'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    return (url)

# url_selection(properties_to_look_for)

In [164]:
# MULTIPLE PAGES:

url = url_selection(properties_to_look_for)
    
count_number = 0

permalink_list = []
titles_list = []
price_list = []
bedrooms_number_list = []
full_bathrooms_list = []
thumbnail_list = []
announcement_id_list = []
address_street_list = []
colonia_list = []
city_list = []
state_list = []
property_type_list = []
type_of_operation_list = []
currency_list = []
real_state_agency_list = []

properties_feature_list = []  
        

executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

browser.visit(url)
browser.is_element_not_present_by_id('list-card-container', wait_time=2)
html = browser.html
soup = bs(html, "html.parser")

number_of_results = soup.find("h1", class_ = "list-result-title").text.split(" ")[0].replace(',','')
number_of_results = int(number_of_results)

total_paging_times = math.ceil(number_of_results/24) + 1
print(F'properties_to_look_for: {properties_to_look_for}, total_paging_times: {total_paging_times}')
print('')


for i in range(1,total_paging_times):   

    
#   HERE IS WHERE I PUT THE URL
    if i == 1:
        url = url
    else:
        if properties_to_look_for == 'apartments':
            url = f'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

        elif properties_to_look_for == 'houses':
            url = f'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    
    
    print(f'URL VISITED: {url}')
    print('')
    
#     browser.visit(url)
#     browser.is_element_not_present_by_id('list-card-container', wait_time=2)
#     html = browser.html
#     soup = bs(html, "html.parser")
    
#     publications_list = soup.find_all("div", class_ = "postingCard")

#     for x in range(len(publications_list)):

#         count_number += 1

#     #     PROPERTY_TYPE:

#         if 'departamento' in url:
#             property_type = 'Departamento'
#         elif 'casa' in url:
#             property_type = 'Casa'

#         property_type_list.append(property_type)

#     #     TYPE OF OPERATION:

#         if 'venta' in url:
#             type_of_operation = 'Venta'
#         elif 'renta' in url:
#             type_of_operation = 'Renta'

#         type_of_operation_list.append(type_of_operation)

#     #   PROPERTY_ID
#         property_id = publications_list[x]["data-id"]
#         announcement_id_list.append(property_id)


#     #     HYPERLINK    
#         hyperlink = f'https://www.inmuebles24.com{publications_list[x]["data-to-posting"]}'
#         permalink_list.append(hyperlink)

#     #     ------SUB_SOUP-----

#         sub_soup = publications_list[x]

#     #     PROPERTY_TITLE
#         ad_title = sub_soup.find_all("h2", class_ = "postingCardTitle")[0].get_text(strip=True)
#         titles_list.append(ad_title)

#     #     PROPERTY PRICE
#         property_price = sub_soup.find_all("span", class_ = "firstPrice")[0].get_text(strip=True).replace('MN ', '').replace(',', '')
#         price_list.append(property_price)

#     #     CURRENCY:

#         if 'USD' in property_price:
#             currency = 'USD'
#         else:
#             currency = 'MXN'

#         currency_list.append(currency)

#     # #     print(f'id: {property_id} , title: {ad_title} , {hyperlink} , price: {property_price}')

#     #     THUMBNAIL:

#         try:
#             thumbnail = sub_soup.find_all("img")[0]['data-src'].strip()
#             thumbnail_list.append(thumbnail)
#         except:
#             thumbnail = sub_soup.find_all("img")[0]['src'].strip()
#             thumbnail_list.append(thumbnail)


#         #   LOCATION TITLE

#         street = sub_soup.find_all("span", class_ = "postingCardLocationTitle")[0].get_text(strip=True)

#         address = sub_soup.find_all("span", class_ = "postingCardLocation")[0].get_text(strip=True).split(",")
#         colonia = address[0]
#         city = address[1]
#         state = 'Distrito Federal'

#         address_street_list.append(street)
#         colonia_list.append(colonia)
#         city_list.append(city)
#         state_list.append(state)

#     #     REAL ESTATE AGENCY:
        
#         try:
#             real_state_agency = sub_soup.find_all("div", class_ = "publisherLogo")[0].a.img['src'].split("/")[-1]

#             if real_state_agency == 'no-logo.svg':
#                 real_state_agency = 'trato_directo'
#             else:
#                 real_state_agency = real_state_agency.replace('.jpg', '').replace('logo_', '')
        
#         except:
#             real_state_agency = 'not_available'

#         real_state_agency_list.append(real_state_agency)

#     #     FEATURES LIST:

#         inside_list = []

#         features_list = sub_soup.find_all("ul", class_ = "postingCardMainFeatures")[0].find_all("li")

#         for j in range(len(features_list)):
#             feature = features_list[j].get_text(strip = True).replace("\n", " ").replace("\t", "")
#             inside_list.append(feature)

#     #         BEDROOM NUMBER:

#             if 'rec.' in feature:
#                 bedrooms_number = feature[0:2]
#                 bedrooms_number_list.append(bedrooms_number)

#     #         BATHROOMS NUMBER:

#             if 'baños' in feature or 'baño' in feature:
#                 bathrooms_number = feature[0:2]
#                 full_bathrooms_list.append(bathrooms_number)

#         if len(full_bathrooms_list) != count_number:
#             full_bathrooms_list.append("not_available")

#         properties_feature_list.append(inside_list)

#         print(f"""ID: {count_number} - property_id: {property_id},
#         hyperlink: {hyperlink}
#         """)
        
#     time.sleep(2.5)

#     #   street: {street}, colonia: {colonia}, city: {city}, state: {state},      
# #     ad_title: {ad_title}, property_price: {property_price},
#     #     thumbnail: {thumbnail} bedrooms_number: {bedrooms_number}, bathrooms_number: {bathrooms_number},
#     #     real_state_agency: {real_state_agency}

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Intel NUC\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


 
properties_to_look_for: apartments, total_paging_times: 77

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-2-q-ciudad-de-mexico.html

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-3-q-ciudad-de-mexico.html

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-4-q-ciudad-de-mexico.html

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-5-q-ciudad-de-mexico.html

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-di

In [138]:
properties_24_df = pd.DataFrame({
    'ad_id': announcement_id_list,
    'title': titles_list,
    'type_of_operation': type_of_operation_list,
    'property_type': property_type_list,
    'price': price_list,
    "currency" : currency_list,
    'bedrooms_number': bedrooms_number_list,
    'full_bathrooms_number': full_bathrooms_list,
    'real_state_agency': real_state_agency_list,
    'permalink': permalink_list,
    'thumbnail': thumbnail_list,
    'address_street': address_street_list,
    'neighborhood': colonia_list,
    'city': city_list,
    'state': state_list,
})

properties_24_df.head()

,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state
0,52913725,Bosque 6060,Venta,Departamento,11312978,MXN,3,not_available,abilia-creando-la-diferencia_1617205333633,https://www.inmuebles24.com/propiedades/bosque...,https://img10.naventcdn.com/listado/RPLISv6.16...,Desierto de los Léones 6060 Ciudad de México,Olivar de los Padres,Alvaro Obregón,Distrito Federal
1,54456938,Sky Tower Lindavista,Venta,Departamento,3210000,MXN,2,not_available,mp-inmobiliaria_4,https://www.inmuebles24.com/propiedades/sky-to...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Av. Insurgentes Norte. 1346, Lindavista, Ciud...",Lindavista,Gustavo A. Madero,Distrito Federal
2,55534814,"Último Departamento, Departamento en Venta Tla...",Venta,Departamento,7950000,MXN,3,not_available,emasn-desarrollos_6,https://www.inmuebles24.com/propiedades/ultimo...,https://img10.naventcdn.com/listado/RPLISv6.16...,Club Alemán 37,Arenal Tepepan,Tlalpan,Distrito Federal
3,60679141,Chicle 160,Venta,Departamento,2052000,MXN,2,not_available,arturo-yanez_1596136396394,https://www.inmuebles24.com/propiedades/chicle...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Chicle 160, Granjas México, Iztacalco, 08400 C...",Granjas México,Iztacalco,Distrito Federal
4,56515561,Up Santa Fe,Venta,Departamento,3701095,MXN,1,not_available,desarrolladora-del-parque_1572280655326,https://www.inmuebles24.com/propiedades/up-san...,https://img10.naventcdn.com/listado/RPLISv6.16...,"Federal México-Toluca 3009, El Yaqui, 05320 Ci...",Santa Fe,Alvaro Obregón,Distrito Federal


In [139]:
properties_24_df.shape

(48, 15)

In [140]:
properties_24_df.title

0                                           Bosque 6060
1                                  Sky Tower Lindavista
2     Último Departamento, Departamento en Venta Tla...
3                                            Chicle 160
4                                           Up Santa Fe
5         Departamento en Pennsylvania Ciudad de México
6             Departamento en Progreso Ciudad de México
7               Departamento en Tanana Ciudad de México
8     Venta de Departamento en Polanco, Ciudad de Mé...
9      Se Vende Departamento en Lomas, Ciudad de México
10    Departamento en Venta en La Colonia del Valle,...
11    Departamento en Venta en Una de Las Mejores Co...
12    Departamentos en Venta en Calle Amores, Ciudad...
13    Departamento en Venta, en Benito Juárez Ciudad...
14    Departamento en Venta Guadalupe, Tlalpan, Ciud...
15    Departamento en Venta, Masaryk Polanco, Ciudad...
16    Departamento Nuevo en Venta Al Sur en La Ciuda...
17    Vendo Departamento Colonia Polanco Ciudad 

In [141]:
properties_24_df[properties_24_df.duplicated(subset=['ad_id'])]


,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state


In [ ]:
browser.quit()

### DEPARTAMENTO QUERY

In [23]:
# DEPARTAMENTO QUERY:

url = 'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
url

'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'

### Debugger code

In [117]:
fields = [announcement_id_list,
    titles_list,
    type_of_operation_list,
    property_type_list,
    price_list,
    currency_list,
    bedrooms_number_list,
    full_bathrooms_list,
    real_state_agency_list,
    permalink_list,
    thumbnail_list,
    address_street_list,
    colonia_list,
    city_list,
    state_list]


for x in range(len(fields)):
    print(f'field {x+1}, length: {len(fields[x])}')

field 1, length: 48
field 2, length: 48
field 3, length: 48
field 4, length: 48
field 5, length: 48
field 6, length: 48
field 7, length: 48
field 8, length: 48
field 9, length: 48
field 10, length: 48
field 11, length: 48
field 12, length: 48
field 13, length: 48
field 14, length: 48
field 15, length: 48


### Analysis

In [92]:
properties_24_df['price'].unique()

array(['11312978', '3210000', '7950000', '2052000', '3701095', '4480000',
       '3650000', '13260000', '11290000', '4300000', '12499000',
       '4844237', '1417500', '4280000', '7200000', '2390000', '15500000',
       '4900000', '4400000', '2598989', '5500000', '2980000'],
      dtype=object)

In [15]:
properties_feature_list

[['18 deptos.', '3 rec.', '12 a 66   m²'],
 ['184 deptos.', '2 rec.', '75 a 169   m²'],
 ['7 deptos.', '3 rec.', '335   m²'],
 ['40 deptos.', '2 rec.', '56 a 57   m²'],
 ['177 deptos.', '1 a 2 rec.', '55 a 84   m²'],
 ['123 m²', '4 rec.', '2  baños', '1  estac.'],
 ['102 m²', '3 rec.', '2  baños', '1  estac.'],
 ['82 m²', '3 rec.', '2  baños', '2  estac.'],
 ['155 m²', '3 rec.', '2  baños', '2  estac.'],
 ['290 m²', '3 rec.', '3  baños', '3  estac.'],
 ['103 m²', '3 rec.', '2  baños', '1  estac.'],
 ['328 m²', '3 rec.', '3  baños', '4  estac.'],
 ['87 m²', '2 rec.', '2  baños', '2  estac.'],
 ['41 m²', '2 rec.', '1  baño'],
 ['25 m²', '2 rec.', '2  baños', '2  estac.'],
 ['136 m²', '2 rec.', '2  baños', '1  estac.'],
 ['63 m²', '2 rec.', '2  baños', '1  estac.'],
 ['192 m²', '3 rec.', '2  baños', '3  estac.'],
 ['92 m²', '3 rec.', '2  baños', '2  estac.'],
 ['55 m²', '2 rec.', '1  baño', '1  estac.'],
 ['901 m²', '3 rec.', '2  baños', '2  estac.'],
 ['80 m²', '2 rec.', '2  baños', '1  

In [16]:
properties_feature_list

[['18 deptos.', '3 rec.', '12 a 66   m²'],
 ['184 deptos.', '2 rec.', '75 a 169   m²'],
 ['7 deptos.', '3 rec.', '335   m²'],
 ['40 deptos.', '2 rec.', '56 a 57   m²'],
 ['177 deptos.', '1 a 2 rec.', '55 a 84   m²'],
 ['123 m²', '4 rec.', '2  baños', '1  estac.'],
 ['102 m²', '3 rec.', '2  baños', '1  estac.'],
 ['82 m²', '3 rec.', '2  baños', '2  estac.'],
 ['155 m²', '3 rec.', '2  baños', '2  estac.'],
 ['290 m²', '3 rec.', '3  baños', '3  estac.'],
 ['103 m²', '3 rec.', '2  baños', '1  estac.'],
 ['328 m²', '3 rec.', '3  baños', '4  estac.'],
 ['87 m²', '2 rec.', '2  baños', '2  estac.'],
 ['41 m²', '2 rec.', '1  baño'],
 ['25 m²', '2 rec.', '2  baños', '2  estac.'],
 ['136 m²', '2 rec.', '2  baños', '1  estac.'],
 ['63 m²', '2 rec.', '2  baños', '1  estac.'],
 ['192 m²', '3 rec.', '2  baños', '3  estac.'],
 ['92 m²', '3 rec.', '2  baños', '2  estac.'],
 ['55 m²', '2 rec.', '1  baño', '1  estac.'],
 ['901 m²', '3 rec.', '2  baños', '2  estac.'],
 ['80 m²', '2 rec.', '2  baños', '1  

In [38]:
properties_24_df.shape

(24, 7)

In [12]:
features_list = sub_soup.find_all("ul", class_ = "postingCardMainFeatures")[0].find_all("li")

additional_list = []

for x in range(len(features_list)):
    test = features_list[x].get_text(strip = True).replace("\n", " ").replace("\t", "")
    additional_list.append(test)
#     print(test)

additional_list

['18 deptos.', '3 rec.', '12 a 66   m²']

## Scrape of each webpage

In [221]:
# THIS SHOULD BE OBTAINED FROM EACH PAGES' WEBSITE:

square_meters_list = []
unit_list = []

item_condition = []

if 'm²' in feature:
    square_meters_list.append(feature.replace("m²", ""))
    unit_list.append('m²') 

In [14]:
import time

image_details_coordinates_list = []
latitude_list = []
longitude_list = []

count_number = 0

for x in range(3):
    
    count_number += 1

    url = permalink_list[x]

    browser.visit(url)

    time.sleep(1)

    html = browser.html

    soup = bs(html, "html.parser")
    
#     SCRAPE OF IMAGE COORDINATES:

    map_image = soup.find('div', class_ = 'article-map')

    image_details = map_image.img['src'].split("&")

    print(permalink_list[x])
    
    for x in range(len(image_details)):
        if 'markers' in image_details[x]:
            image_details_coordinates = image_details[x].replace('markers=', '').split(",")
            image_details_coordinates_list.append(image_details_coordinates)
            
            latitude = image_details_coordinates[0]
            longitude = image_details_coordinates[1]            
            latitude_list.append(latitude)
            longitude_list.append(longitude)
                        
    if len(image_details_coordinates_list) != count_number:
        for x in range(len(image_details)):
            if 'center' in image_details[x]:
                image_details_coordinates = image_details[x].replace('center=', '').split(",")
                image_details_coordinates_list.append(image_details_coordinates)

                latitude = image_details_coordinates[0].split("?")[1]
                longitude = image_details_coordinates[1]            
                latitude_list.append(latitude)
                longitude_list.append(longitude)

    print(latitude, longitude)

https://www.inmuebles24.com/propiedades/bosque-6060-52913725.html
19.331155400000000 -99.256939199999980
https://www.inmuebles24.com/propiedades/sky-tower-lindavista-54456938.html
19.481846100000000 -99.129833500000020
https://www.inmuebles24.com/propiedades/ultimo-departamento-departamento-en-venta-tlalpan-55534814.html
19.279131400000000 -99.140570900000000


In [261]:
pd.DataFrame({
    "bedrooms_number" : bedrooms_number_list
})

,bedrooms_number
0,3
1,2
2,3
3,2
4,1
5,2
6,4
7,3
8,3
9,2


In [126]:
browser.quit()